In [ ]:
%load_ext autoreload
import numpy as np
import h5py as h5
import pandas as pd

%autoreload 2
from zcu_tools.analysis.fluxdep import calculate_energy, VisualizeSpet

# Load Parameters

In [ ]:
qub_name = "S001"

loadpath = f"../result/{qub_name}/params.csv"
df = pd.read_csv(loadpath)

cflx, period = df.loc[0, ["Half flux", "Period"]]
params = df.loc[0, ["EJ", "EC", "EL"]].values

df.head()

In [ ]:
def phi2mA(ratio):
    return (period * (ratio - 0.5) + cflx) * 1e3


flxs = np.linspace(0.0, 1.0, 2000)
mAs = phi2mA(flxs)

# Load Points & Spectrums

In [ ]:
# loading points
r_f = 7.48742

df = pd.read_csv(
    f"../result/{qub_name}/freqs.csv", usecols=["Current (mA)", "Freq (MHz)", "T1 (μs)"]
)
s_mAs = df["Current (mA)"].values  # mA
s_freqs = df["Freq (MHz)"].values * 1e-3  # GHz
s_T1s = df["T1 (μs)"].values

In [ ]:
load_path = f"../result/{qub_name}/spect.hdf5"

s_spects = {}
with h5.File(load_path, "r") as f:
    for key in f.keys():
        grp = f[key]
        spectrum = (
            grp["spectrum"]["flxs"][()],
            grp["spectrum"]["fpts"][()],
            grp["spectrum"]["data"][()],
        )
        points = (grp["points"]["flxs"][()], grp["points"]["fpts"][()])
        s_spects.update(
            {
                key: {
                    "cflx": grp["cflx"][()] * 1e3,
                    "spectrum": spectrum,
                    "points": points,
                }
            }
        )
s_spects.keys()

# Align cflx

In [ ]:
t_cflx = list(s_spects.values())[-1]["cflx"]
for spect in s_spects.values():
    shift = t_cflx - spect["cflx"]
    spect["cflx"] += shift
    spect["spectrum"] = (phi2mA(spect["spectrum"][0] + shift), *spect["spectrum"][1:])
    spect["points"] = (phi2mA(spect["points"][0] + shift), *spect["points"][1:])

# Plot Map

In [ ]:
energies = calculate_energy(flxs, *params, 50, 15)

In [ ]:
r_f = 7.48742
allows = {
    # "transitions": [(0, 1)],
    "transitions": [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3)],
    "red side": [(0, 4), (0, 3), (1, 5)],
    "r_f": r_f,
}

In [ ]:
vs = VisualizeSpet(s_spects, s_mAs, s_freqs, mAs, energies, allows)
vs.set_scatter_style(10, color=s_T1s)
fig = vs.create_figure()
fig.add_hline(y=r_f, line_dash="dash", line_color="black", name="Cavity freq")
fig.add_vline(x=cflx * 1e3, line_dash="dash", line_color="black", name="Cavity freq")
fig.update_layout(
    title_text=f"EJ/EC/EL = {params[0]:.2f}/{params[1]:.2f}/{params[2]:.2f}",
    showlegend=True,
)

fig.show()

# save to html

In [ ]:
save_path = f"../result/{qub_name}/web/flux_dep.html"
fig.write_html(save_path, include_plotlyjs="cdn")